# Math 266 - Project #6 - Singular value decomposition

Execute each cell below and follow the instructions.

In [ ]:
import numpy as np
import numpy.linalg as la
from numpy import cos,sin,pi
import matplotlib.pyplot as plt
from numpy.linalg import norm
import imageio

import warnings
warnings.filterwarnings('ignore')

from sympy import *
from sympy.interactive.printing import init_printing
init_printing(use_unicode=False, wrap_line=False)


---
# The singular value decomposition for a matrix A is given by:
 $$A = U \Sigma V^T$$
 
 Where U and V are unitary matrices. $\Sigma$ is a "diagonal matrix containing the singular values for A. The singular values $\sigma_i$ are in descending order in matrix $\Sigma$ .

---

# Example from text.  Construct a Singular Value Decomposition of the matrix A,  p. 443 in text.

$$ A=\begin{bmatrix}4 & 11 & 14 \\8 & 7 & -2 \end{bmatrix}\ $$

$$A = U \Sigma V^{T}$$


---
### To construct the singular value for the matrix A the text outlines the following steps.

Step 1. Find the orthogonal diagonalization of $A^{T}A$.

step 2. Form matrix V from the eigenvectors for $A^{T}A$.

step 3. The singular values are the square root of eigenvalues of $A^{T}A$.<br> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp; Put the values in descending order and form the matrix $\Sigma$.

step 4. Finally form the matrix U.  Column i of U is given by:    $\vec{\mathbf{u_i}}=A\vec{\mathbf{v_i}}/\sigma_i$

---


### Using python we can find the singular value decomposition for matrix A with the simple command:
``` python
np.linalg.svd(A)

```
---
---

Execute the cell below to create matrix A.

In [ ]:
A = np.array([4,11,14,8,7,-2]).reshape(2,3)
print ("A=",end="");Matrix(A) # command to simply display matrix in nice format

___
Execute cell below to compute the singular value decomposition of A and view the results. Compare the result to page 444 in text.

___

In [ ]:
u,s,v = la.svd(A)  # easier way to do it

[Matrix(u),Matrix(np.diag(s)),Matrix(v)]

Now we will confirm that $A = U \Sigma V^{T}$ .  Note the svd command returns an array s of singular values and we need to create the diagonal matrix $ \Sigma $ with the command:
```python
np.diag(s)

```
Now $\Sigma$ must be the same size as A so we need to append a column of zeros to make $\Sigma$ size (3,2). <br>

We do this with the command:

```python
np.c_[np.diag(s),[0,0]]

```
---

Execute the cells below to confirm the decomposition. That is that  $A = U \Sigma V^{T}$ 

In [ ]:
sigma=np.c_[np.diag(s),[0,0]]
print("sigma = ");Matrix(sigma)

In [ ]:
print(u@sigma@v) # confirm the decomposition - see the output below

---
---

# Application 1.    Image compression

Recall that a "grayscale" image is just an rectangular array (matrix) of grayscale values for each pixel.

Th rank n approximation of an Image is given by:

$$A = \sum_{i=0}^{n-1}\sigma_i\vec{\mathbf{u_i}}\vec{\mathbf{v_i}}^{T} $$

---

---

Execute the following cells to import the test image and display it. This is an image of the mathematician Bernhare Reimann. We store the image <br>
in a (246,225) matrix and call it "img". 

---

In [ ]:
img = imageio.imread("https://github.com/rmartin977/Math-266/blob/main/rieman.jpeg?raw=true")

In [ ]:
img.shape

In [ ]:
plt.imshow(img,cmap='gray')

---
Execute the following to compute SVD of image.

---

In [ ]:
U,S,V = la.svd(img)

---
Below we do a plot of the magnitudes of the singular values. Notice how they fall off rapidly. This tells us a low rank approximation (image compression) will be good.

---

In [ ]:
plt.plot(S)
plt.grid()
plt.title("singular values");


---

Execute the cells below to see a rank 1 approximation for our image.

---

In [ ]:
rank_1 = S[0]*np.outer(U[:,0],V[0,:])
rank_1.shape

In [ ]:
plt.imshow(rank_1,cmap='gray')

Below we define a helper function that will compute a rank_k approximation for our images. The function basically computes the partial sum:

$$ \sum_{i=0}^{k-1}\sigma_i\vec{\mathbf{u_i}}\vec{\mathbf{v_i}}^{T} $$

In [ ]:
def rank(k):
    '''
    The function will return a rank_k approximation for the image of Rieman.
    '''
    return np.sum([ S[i]*np.outer(U[:,i],V[i,:]) for i in range(k)],axis=0)

The cell below computes a rank 5 approximation of the image matrix and the result is displayed. Already starting to look like Bernhard. Execute the cell to output.  Change the rank and oberve difference in output.  Try rank = 50.

In [ ]:
plt.imshow(rank(5),cmap='gray')  # rank 5  change the value to see different rank approximations.

---

Below we create a rank-5 approximation and store it in matrix Berhnard_5 and save image to file.

---

In [ ]:
Bernhard_5 = rank(5)

In [ ]:
plt.axis('off')
plt.imshow(rank_5,cmap='gray')
plt.savefig("Bernhard_5.jpg") # you will need to change path to file

# to save on your G-drive you need to:
# 1. Mount your drive.
# 2. Change the path to the file as "drive/MyDrive/Bernhard_5.jpg"
# 3. Contact me if you need help. Verify that your image is saved and you can display it outside of google colab.

---
---

# Your turn. 

Import the image of MonaLisa and creat a rank_5 approximation for this image.  Save a copy of the image to your G-drive.  You will need to upload the image to Gradescope later.

---
---

In [ ]:
image = imageio.imread("https://github.com/rmartin977/Math-266/blob/main/ml.jpg?raw=true")

In [ ]:
plt.imshow(image,cmap='gray')

In [ ]:
#Enter your code here to create a rank-5 approximation of monalisa and save the jpeg to your G Drive.  Name the file monalisa_5.jpg .

---
---
# Application #2. Pseudo inverse.

We will find a least square solution for the system from exam #3. $$ \begin{bmatrix}1 & 1  \\1 & 2\\1 & 3\end{bmatrix}\ \begin{bmatrix}\beta_0  \\ \beta_1  \end{bmatrix}= \begin{bmatrix}0  \\ 1\\1  \end{bmatrix} $$ 

By computing the pseudo inverse of matrix A.  A non square matrix does not have a true inverse however we can compute a pseudo inverse by taking the inverse of the "reduced" singular value decomposition of A.  This inverse is denoted $A^t$ and is computed as follows.

$$A^{t} = V\Sigma ^{-1} U^{T}$$



---
---

---
First we recompute the svd for A, and then we comptute the pseudo inverse. We will call the pseduo inverse A_pinv. Execute the cells below.

___


Define A and b

In [ ]:
A = np.array([1,1,1,2,1,3]).reshape(3,2)
b = np.array([0,1,1])
Matrix(A),Matrix(b)

Compute pseudo inverse.  See 7.4 in the text.

In [ ]:
u,s,v = la.svd(A,full_matrices=False) # compute reduced svd see 7.4 in text
sigma_inv = np.diag(1/s)
A_pinv = v.T@sigma_inv@u.T
Matrix(A_pinv)

Numpy has a command to compute the pseudo inverse directly.  The command is la.pinv(A).  Execute the cell below and compare.

In [ ]:
la.pinv(A)

---

Finally let us use A_inv to solve system. Execute the cell below.

___

In [ ]:
x_hat = A_pinv @ b
Matrix(x_hat)

---
Compare to solution given by least square.

---

In [ ]:
la.lstsq(A,b)[0]

---

Compare to pseudoinverse to pseudoinverse given by numpy.

---

## Your turn...
Answer the following 4 questions.  Go to gradescope Lecture #10 and enter your answers.

1. Upload your rank 5 image of MonaLisa.
2. What is the largest singular value for the matrix $A= \begin{bmatrix}1 & 2 & 1 \\ 1 & 1 & -1 \end{bmatrix}$?
3. Use python to solve the system using pseudo inverse. You can use la.pinv.  $$ \begin{bmatrix}1 & 2 & 1 \\1 & 1 & -1 \end{bmatrix}\ \begin{bmatrix}x_1  \\ x_2\\x_3  \end{bmatrix}= \begin{bmatrix}1  \\ 1  \end{bmatrix} $$ 
4. The image of Bernhard Reimann is (246,225) pixels.  So there are 246 * 225 = 55,350 values necessary to store or transmit the image.  How many values are necessary for the rank 5 approximation? <br> Hint: U.shape=(246,246) and V.shape = (225,225).  Also the answer is less than 5000.